In [15]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
%pwd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/ocean/projects/asc170022p/mtragoza/lung-project'

In [16]:
import sys, os
import numpy as np
import xarray as xr
%aimport data
%aimport register
%aimport utils

In [17]:
case = data.Emory4DCTCase(data_root='data/Emory-4DCT', case_name='Case1Pack', phases=[0, 50])
case.load_niftis()
case

Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T00.nii.gz
Loading data/Emory-4DCT/Case1Pack/NIFTI/case1_T50.nii.gz


Emory4DCTCase('data/Emory-4DCT', 'Case1Pack', 2 phases)

In [18]:
case.view(groupby=['phase', 'y'], x='x', y='z')

:DynamicMap   [phase,y]
   :Image   [x,z]   (case1)

In [19]:
%autoreload
reg = register.Registration()
F = reg.execute(
    fixed=case.array.sel(phase=0),
    moving=case.array.sel(phase=50),
    type='bspline'
)

RegistrationParameterScalesFromPhysicalShift (0x5652fcc56820): Variation in any parameter won't change a voxel position. The default scales (1.0) are used to avoid division-by-zero.

RegistrationParameterScalesFromPhysicalShift (0x5652fcc56820): Variation in any parameter won't change a voxel position. The default scales (1.0) are used to avoid division-by-zero.

RegistrationParameterScalesFromPhysicalShift (0x5652fcc56820): Variation in any parameter won't change a voxel position. The default scales (1.0) are used to avoid division-by-zero.

RegistrationParameterScalesFromPhysicalShift (0x5652fcc56820): Variation in any parameter won't change a voxel position. The default scales (1.0) are used to avoid division-by-zero.

RegistrationParameterScalesFromPhysicalShift (0x5652fcc56820): Variation in any parameter won't change a voxel position. The default scales (1.0) are used to avoid division-by-zero.

RegistrationParameterScalesFromPhysicalShift (0x5652fcc56820): Variation in any param

Estimated scales:  (1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)
[   0] 11068.6821904

In [22]:
%%time

spatial_shape = [case.array.sizes[d] for d in 'xyz']
spatial_coords = [case.array.coords[d] for d in 'xyz']

input_points = utils.nd_coords(spatial_coords)
output_points = F.apply(input_points)
U = output_points - input_points

displacement = xr.DataArray(
    U.reshape(spatial_shape + [3]),
    dims=['x', 'y', 'z', 'component'],
    coords={
        'x': case.array.x,
        'y': case.array.y,
        'z': case.array.z,
        'component': ['x', 'y', 'z']
    }
)
displacement

CPU times: user 10.8 s, sys: 626 ms, total: 11.4 s
Wall time: 12.8 s


<xarray.DataArray (x: 256, y: 256, z: 94, component: 3)>
array([[[[0.68501202, 2.42123419, 0.93935207],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        [[0.47046692, 0.79466423, 0.39283781],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         ...,
...
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]],

        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]]]])
Coordinates:
  * x          (x) float64 0.0 0.97 1.94 2.91 3.88 ... 244.4 245.4 246.4 247.4
  * y          (y) float64 0.0 0.97 1.94 2.91 3.88 ... 244.4 245.4 246.4 247.4
  * z          (z) float64 0.0 2.5 5.0 7.5 10.0 ... 225.0 227.5 230.0 232.5
  * component  (component) <U1 'x' 'y' 'z'

In [23]:
data.view_array(displacement, groupby=['y', 'component'], x='x', y='z')

:DynamicMap   [y,component]
   :Image   [x,z]   (value)

In [28]:
import SimpleITK as sitk
dict(sitk.GetDefaultParameterMap('bspline'))

{'AutomaticParameterEstimation': ('true',),
 'CheckNumberOfSamples': ('true',),
 'DefaultPixelValue': ('0.0',),
 'FinalBSplineInterpolationOrder': ('3',),
 'FinalGridSpacingInPhysicalUnits': ('8.000000',),
 'FixedImagePyramid': ('FixedSmoothingImagePyramid',),
 'GridSpacingSchedule': ('2.803221', '1.988100', '1.410000', '1.000000'),
 'ImageSampler': ('RandomCoordinate',),
 'Interpolator': ('LinearInterpolator',),
 'MaximumNumberOfIterations': ('256',),
 'MaximumNumberOfSamplingAttempts': ('8',),
 'Metric': ('AdvancedMattesMutualInformation',
  'TransformBendingEnergyPenalty'),
 'Metric0Weight': ('1.0',),
 'Metric1Weight': ('1.0',),
 'MovingImagePyramid': ('MovingSmoothingImagePyramid',),
 'NewSamplesEveryIteration': ('true',),
 'NumberOfResolutions': ('4',),
 'NumberOfSamplesForExactGradient': ('4096',),
 'NumberOfSpatialSamples': ('2048',),
 'Optimizer': ('AdaptiveStochasticGradientDescent',),
 'Registration': ('MultiMetricMultiResolutionRegistration',),
 'ResampleInterpolator': ('Fin

In [29]:
dict(sitk.GetDefaultParameterMap('rigid'))

{'AutomaticParameterEstimation': ('true',),
 'AutomaticScalesEstimation': ('true',),
 'CheckNumberOfSamples': ('true',),
 'DefaultPixelValue': ('0.0',),
 'FinalBSplineInterpolationOrder': ('3',),
 'FixedImagePyramid': ('FixedSmoothingImagePyramid',),
 'ImageSampler': ('RandomCoordinate',),
 'Interpolator': ('LinearInterpolator',),
 'MaximumNumberOfIterations': ('256',),
 'MaximumNumberOfSamplingAttempts': ('8',),
 'Metric': ('AdvancedMattesMutualInformation',),
 'MovingImagePyramid': ('MovingSmoothingImagePyramid',),
 'NewSamplesEveryIteration': ('true',),
 'NumberOfResolutions': ('4',),
 'NumberOfSamplesForExactGradient': ('4096',),
 'NumberOfSpatialSamples': ('2048',),
 'Optimizer': ('AdaptiveStochasticGradientDescent',),
 'Registration': ('MultiResolutionRegistration',),
 'ResampleInterpolator': ('FinalBSplineInterpolator',),
 'Resampler': ('DefaultResampler',),
 'ResultImageFormat': ('nii',),
 'Transform': ('EulerTransform',),
 'WriteIterationInfo': ('false',),
 'WriteResultImage':

In [34]:
method = sitk.ImageRegistrationMethod()
method.SetMetricAsMeanSquares()
method.SetInterpolator(sitk.sitkLinear)
method.SetOptimizerAsAdaptiveStochasticGradientDescent()

AttributeError: 'ImageRegistrationMethod' object has no attribute 'SetOptimizerAsAdaptiveStochasticGradientDescent'

In [35]:
filter = sitk.ElastixImageFilter()